In [ ]:
!pip install -q pymongo llamaapi

In [ ]:
cd /content/drive/MyDrive/Mumbai-Hackathon

/content/drive/MyDrive/Mumbai-Hackathon


In [ ]:
# from urllib.request import urlopen
# import re as r

# def getIP():
#     d = str(urlopen('http://checkip.dyndns.com/').read())
#     return r.compile(r'Address: (\d+\.\d+\.\d+\.\d+)').search(d).group(1)

# print(getIP())

34.148.5.18


In [ ]:
import json
from llamaapi import LlamaAPI

llama = LlamaAPI()

def prompt_llama(data):
  output_format = {
      "product_id": 3454475,
      "closet_id": ['professional_attire, semi_professional attire']
  }
  api_request_json = {
      "model": "llama3.1-405b",
      "messages": [
          {"role": "user", "content": f"""
          [<Role>]You role is of a fashion expert.[/<Role>]
          [<data>]This is catalog data: {data}[/<data>]
          [<Instruction>]You have to cluster products on the basis of product attributes provided. The products are to be clustered on following instructions:
          1. Each product is to be clustered as it is to be putted in separate closet.
          2. list of closet_id is [gym_wear, family_party_wear, friends_party_wear, night_club, profesional_attire, semi_professional attire, traditional].
          2. One product can belong to multiple clusters but the closet_id are fixed as mentioned above.
          3. Clustering should be extremely mindful you should NOT make strange combinations of closet_id like mixing traditional and professional.
          4. Result should be in following format. {output_format}
          5. Donot provide code perform the data clustering and return the result[/<Instruction>]
          """},
      ],
      "stream": False,
  }
  response = llama.run(api_request_json)

  return response.json()

batch_res = []
for batch in range(0,len(prod_li_filtered),20):
  batch_data = prod_li_filtered[batch:batch+20]
  r = prompt_llama(batch_data)
  print(batch_data)
  batch_res.append(r.get("choices")[0]["message"]["content"])




# from urllib.request import urlopen
# import re as r

# def getIP():
#     d = str(urlopen('http://checkip.dyndns.com/').read())
#     return r.compile(r'Address: (\d+\.\d+\.\d+\.\d+)').search(d).group(1)

# print(getIP())


# from llama_index.indices.multi_modal.base import MultiModalVectorStoreIndex
# from llama_index import SimpleDirectoryReader, StorageContext

# from llama_index import SimpleDirectoryReader, StorageContext
# from llama_index.vector_stores import LanceDBVectorStore


# from llama_index import (
#     SimpleDirectoryReader,
# )

# text_store = LanceDBVectorStore(uri="lancedb", table_name="text_collection")
# image_store = LanceDBVectorStore(uri="lancedb", table_name="image_collection")
# storage_context = StorageContext.from_defaults(
#     vector_store=text_store, image_store=image_store
# )

# # Create the MultiModal index
# documents = SimpleDirectoryReader(output_folder).load_data()

# index = MultiModalVectorStoreIndex.from_documents(
#     documents,
#     storage_context=storage_context,
# )
# retriever_engine = index.as_retriever(
#     similarity_top_k=5, image_similarity_top_k=5
# )
# from llama_index.response.notebook_utils import display_source_node
# from llama_index.schema import ImageNode


# def retrieve(retriever_engine, query_str):
#     retrieval_results = retriever_engine.retrieve(query_str)

#     retrieved_image = []
#     retrieved_text = []
#     for res_node in retrieval_results:
#         if isinstance(res_node.node, ImageNode):
#             retrieved_image.append(res_node.node.metadata["file_path"])
#         else:
#             display_source_node(res_node, source_length=200)
#             retrieved_text.append(res_node.text)

#     return retrieved_image, retrieved_textdef retrieve(retriever_engine, query_str):
#     retrieval_results = retriever_engine.retrieve(query_str)
# from llama_index.multi_modal_llms.openai import OpenAIMultiModal

# openai_mm_llm = OpenAIMultiModal(
#     model="gpt-4-vision-preview", api_key=OPENAI_API_TOKEN, max_new_tokens=1500
# )


# response_1 = openai_mm_llm.complete(
#     prompt=qa_tmpl_str.format(
#         context_str=context_str, query_str=query_str, metadata_str=metadata_str
#     ),
#     image_documents=image_documents,
# )

# pprint(response_1.text)

In [ ]:
from pymongo import MongoClient

db_password = ""
connection_string = f""

client = MongoClient(connection_string)

db = client['closet']

collection = db['data_all_1']
documents = collection.find()

prod_li = []
for doc in documents:
  prod_li.append(doc)
cluster_attr = ['product_name', 'product_id', 'product_type','description','created_at', 'neckline', 'sleeve_length', 'categories', 'artists', 'patterns', 'styles', 'material_compositions', 'seasons','closure_types', 'special_features','textures', 'allergens']

prod_li_filtered = []
for prod in prod_li:
  data = {}
  for attr in cluster_attr:
    data[attr] = prod[attr]
  prod_li_filtered.append(data)

In [ ]:
import google.generativeai as genai

genai.configure(api_key='')
model = genai.GenerativeModel('gemini-1.5-flash')

def get_gemini_response(context):
    messages = [{"role": "user", "parts": [context]}]

    response = model.generate_content(messages)

    if response.parts:
        return response.parts[0].text
    else:
        return "Sorry, I couldn't generate a response. Please try again."



data = get_gemini_response("states in India")

In [ ]:
cleaned = data.replace("```json","").replace("```","").replace("\n","")

In [ ]:
import os
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting, FinishReason
import vertexai.generative_models as generative_models

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/hackathon-mumbai/cloudwerx_creds.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/hackathon-mumbai/aiml_creds.json"
# vertexai.init(project="project-magic-dj-poc", location="us-central1")
vertexai.init(project="", location="us-central1")

def generate(context, data):

  model = GenerativeModel(
    "gemini-1.5-flash-001",
    system_instruction=[context]
  )
  generation_config = {
      "max_output_tokens": 8192,
      "temperature": 1,
      "top_p": 0.95,
  }
  responses = model.generate_content(
      [f"This is the product list:\n{data}"],
      generation_config=generation_config
  )
  return responses.text

In [ ]:
context = """You are a fashion expert. Your task is to cluster products based on their attributes into specific closet categories. The clustering must adhere to the following rules:

1. Each product is assigned to one or more clusters, representing the closet categories it belongs to.
2. The available closet categories are: ["gym_wear", "family_party_wear", "friends_party_wear", "night_club", "professional_attire", "semi_professional_attire", "traditional"].
3. Each product may belong to multiple clusters, but avoid strange combinations such as mixing "traditional" and "professional" attire.
4. The output must be in JSON format, following this structure:

{
    "product_id": <product_id>,
    "closet_id": [<closet_category_1>, <closet_category_2>, ...]
}

Example output:
{
    "product_id": 3454475,
    "closet_id": ["professional_attire", "semi_professional_attire"]
}
"""


In [ ]:
import time
batch_res = []
for batch in range(0,len(prod_li_filtered),20):
  print(batch)
  batch_data = prod_li_filtered[batch:batch+20]
  r = generate(context, batch_data)
  print(r.replace("```json","").replace("```","").replace("'", '"').replace("\n",""))
  d = json.loads(r.replace("```json","").replace("```","").replace("'", '"').replace("\n",""))
  batch_res.append(d)
  if batch%100==0:
    time.sleep(30)



0
[  {    "product_id": 984779,    "closet_id": [      "friends_party_wear",      "gym_wear",      "semi_professional_attire"    ]  },  {    "product_id": 989849,    "closet_id": [      "friends_party_wear",      "gym_wear",      "semi_professional_attire"    ]  },  {    "product_id": 757106,    "closet_id": [      "friends_party_wear",      "gym_wear",      "semi_professional_attire"    ]  },  {    "product_id": 530243,    "closet_id": [      "friends_party_wear",      "gym_wear",      "semi_professional_attire"    ]  },  {    "product_id": 170386,    "closet_id": [      "professional_attire",      "semi_professional_attire",      "friends_party_wear"    ]  },  {    "product_id": 197819,    "closet_id": [      "friends_party_wear",      "gym_wear",      "semi_professional_attire"    ]  },  {    "product_id": 969662,    "closet_id": [      "friends_party_wear",      "gym_wear",      "semi_professional_attire"    ]  },  {    "product_id": 926564,    "closet_id": [      "professional_att

In [ ]:
final_res = []
for b in batch_res:
  final_res.extend(b)

In [ ]:
final_res

[{'product_id': 984779,
  'closet_id': ['friends_party_wear', 'gym_wear', 'semi_professional_attire']},
 {'product_id': 989849,
  'closet_id': ['friends_party_wear', 'gym_wear', 'semi_professional_attire']},
 {'product_id': 757106,
  'closet_id': ['friends_party_wear', 'gym_wear', 'semi_professional_attire']},
 {'product_id': 530243,
  'closet_id': ['friends_party_wear', 'gym_wear', 'semi_professional_attire']},
 {'product_id': 170386,
  'closet_id': ['professional_attire',
   'semi_professional_attire',
   'friends_party_wear']},
 {'product_id': 197819,
  'closet_id': ['friends_party_wear', 'gym_wear', 'semi_professional_attire']},
 {'product_id': 969662,
  'closet_id': ['friends_party_wear', 'gym_wear', 'semi_professional_attire']},
 {'product_id': 926564,
  'closet_id': ['professional_attire',
   'semi_professional_attire',
   'friends_party_wear']},
 {'product_id': 987625,
  'closet_id': ['gym_wear', 'friends_party_wear', 'semi_professional_attire']},
 {'product_id': 869907,
  'clo